<a href="https://colab.research.google.com/github/zackives/upenn-cis-2450/blob/main/cis2450lab3nb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CIS 2450 Lab 3: SQL

The goal of this lab is to review SQL and become comfortable with converting between Pandas and SQL syntax.

A relational database is one type of database. It uses a structure that allows us to identify and access data in relation to another piece of data in the database. Data in a relational database is organized into tables.

### Relational Algebra
Relational Algebra is the theoretical framework of working with data stored in a relational model. It forms the theoretical foundation for SQL and relational database operations.

The basic elements of relational algebra include:
- Relations (tables) - The fundamental operands that relational algebra operates on.
- Attributes (columns) - The named data elements that make up relations.
- Tuples (rows) - The individual records in a relation.


Here are the basic operators:
- Selection - Selects a subset of rows from a relation based on a condition. (SQL equivalent: WHERE clause)
- Projection - Selects specific columns from a relation. (SQL equivalent: SELECT clause)
- Union - Combines tuples from two relations, removing duplicates. (SQL equivalent: UNION)
- Set difference - Returns tuples in one relation but not in another. (SQL equivalent: EXCEPT or MINUS)
- Cartesian product - Combines each tuple from one relation with every tuple from another. (SQL equivalent: CROSS JOIN)

Additional common operators:
- Intersection - Returns tuples common to both relations. (SQL equivalent: INTERSECT)
- Join - Combines related tuples from two relations based on a condition. (SQL equivalent: various JOIN types)

We will introduce these abstract concepts through a practical approach below.

### SQL - Structured Query Language

SQL is a language to perform relational algebra operations.

- We write queries in SQL to retrieve data and answer questions about it.
- Declarative Language (not procedural) - You describe what the result you want is, NOT how to obtain the result.

Using an SQL query, you can create and delete, or modify tables, as well as select, insert, and delete data from existing tables.

NOTE: The exact syntax of SQL may vary depending on the underlying database you are using. But most are very similar.


In [ ]:
!pip3 install duckdb

In [ ]:
import duckdb
import numpy as np
import pandas as pd

### Our Dataset

#### Spaceship Management Database

We want to keep track of:
- `crew.csv`: Crew members master file
- `roles.csv`: member and their roles on the spaceship(captain,scientist, etc…)
- `equipment.csv`: Equipment (centrifuge, lab gloves, soldering stations, etc..).
- `worklog.csv`: Hours and what days crew members worked.
- `manages.csv`: Which crew members manage which equipment.

We want to ask questions about this data.

### Import Data

Please download the data onto your local drive and drop them into the folder within this Colab. You can navigate to the folder by selecting the 📁 folder icon in the left sidebar, and drag & drop the csv files into it.

- crew.csv [Download](https://drive.google.com/file/d/1vpUdssqCn9EVn9KGAANTHkc7IkcpwqyN/view?usp=sharing)
- roles.csv [Download](https://drive.google.com/file/d/1x0ASBcsXg7jDtal4726R7I1kSffnI5hB/view?usp=sharing)
- equipment.csv [Download](https://drive.google.com/file/d/172C83HtkP0SjxF_teGgS0i9Ii9jcSPlz/view?usp=sharing)
- worklog.csv [Download](https://drive.google.com/file/d/1wwnbUGAuGQR611qfr2CHIAVyR0NBpFXL/view?usp=sharing)
- manages.csv [Download](https://drive.google.com/file/d/1thRZBRfmyMl4rDdSy9-OlZZxKSmIeS5z/view?usp=sharing)

In [ ]:
crew_df = pd.read_csv("crew.csv")
roles_df = pd.read_csv("roles.csv")
equipment_df = pd.read_csv("equipment.csv")
manages_df = pd.read_csv("manages.csv")
worklog_df = pd.read_csv("worklog.csv")

### Examining the Data

In [ ]:
crew_df.head()

,id,name,rank,role_id
0,1,Jane,10,1.0
1,2,Dan,9,2.0
2,3,Alex,4,3.0
3,4,Jen,4,4.0
4,5,Brandon,1,NaN


In [ ]:
crew_df.dtypes

,0
id,int64
name,object
rank,int64
role_id,float64


In [ ]:
# We can use sql to do this as well
duckdb.sql("SELECT * FROM crew_df LIMIT 5")

┌───────┬─────────┬───────┬─────────┐
│  id   │  name   │ rank  │ role_id │
│ int64 │ varchar │ int64 │ double  │
├───────┼─────────┼───────┼─────────┤
│     1 │ Jane    │    10 │     1.0 │
│     2 │ Dan     │     9 │     2.0 │
│     3 │ Alex    │     4 │     3.0 │
│     4 │ Jen     │     4 │     4.0 │
│     5 │ Brandon │     1 │    NULL │
└───────┴─────────┴───────┴─────────┘

In [ ]:
roles_df.head()

,role_id,name
0,1,captain
1,2,scientist
2,3,engineer
3,4,engineer 2


In [ ]:
roles_df.dtypes

,0
role_id,int64
name,object


In [ ]:
equipment_df.head()

,id,name
0,1,Centrifuge
1,2,Soldering Station
2,3,Notebook
3,4,Chemical Z


In [ ]:
equipment_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      4 non-null      int64 
 1   name    4 non-null      object
dtypes: int64(1), object(1)
memory usage: 192.0+ bytes


In [ ]:
manages_df.head()

,id,crew_id,equip_id
0,1,2,1
1,2,3,2
2,3,1,3
3,4,2,4
4,5,1,4


In [ ]:
manages_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   id        5 non-null      int64
 1   crew_id   5 non-null      int64
 2   equip_id  5 non-null      int64
dtypes: int64(3)
memory usage: 248.0 bytes


In [ ]:
worklog_df.head()

,id,crew_id,day,hours
0,1,1,1,10
1,2,2,1,5
2,3,3,1,8
3,4,4,1,12
4,5,1,2,5


In [ ]:
worklog_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   id       9 non-null      int64
 1   crew_id  9 non-null      int64
 2   day      9 non-null      int64
 3   hours    9 non-null      int64
dtypes: int64(4)
memory usage: 416.0 bytes


Now that we have formed dataframes for our tables, we can use SQL. In this course, we will use [DuckDB](https://duckdb.org/) as a means of managing our tables.

Suppose we just want to list down the names of crew members. SELECT helps use retrieve the rows and columns which we would like to see:

In [ ]:
# Select only the names of crew members
query_crew_names = '''
SELECT name
FROM crew_df
'''
crew_names_df = duckdb.sql(query_crew_names).df()
crew_names_df

,name
0,Jane
1,Dan
2,Alex
3,Jen
4,Brandon


In [ ]:
# In pandas, this is just:
crew_df['name']

,name
0,Jane
1,Dan
2,Alex
3,Jen
4,Brandon


#### Conditional Retrieval

We use the WHERE clause to apply a condition to our retrieval and filter the rows we want.

In [ ]:
# Retrieve all tuples where crew members have rank either 10 or 4 and their name starts with letter J
query_conditional = '''
SELECT *
FROM crew_df
WHERE (rank=10 OR rank=4) AND name LIKE 'J%'
'''
crew_rank_df = duckdb.sql(query_conditional).df()
crew_rank_df

,id,name,rank,role_id
0,1,Jane,10,1.0
1,4,Jen,4,4.0


In [ ]:
# In pandas, we can do the same with the following:
crew_df[(crew_df['rank'] == 10) | (crew_df['rank'] == 4) & (crew_df['name'].str.startswith('J'))]

,id,name,rank,role_id
0,1,Jane,10,1.0
3,4,Jen,4,4.0


#### Ordering

We can order the returned results by values in the columns.

Let’s retrieve the equipment list in increasing order by crew_id, breaking ties with equip_id in decreasing order.

In [ ]:
query_ordering = '''
SELECT *
FROM manages_df
ORDER BY crew_id, equip_id DESC
'''
manages_order_df = duckdb.sql(query_ordering).to_df()
manages_order_df

,id,crew_id,equip_id
0,5,1,4
1,3,1,3
2,4,2,4
3,1,2,1
4,2,3,2


Note that ORDER BY sorts in ascending order by default (ASC), and uses DESC for descending order.

When you order by multiple columns, you can list from highest priority to least. If there is an equal value in a column, the next one in the list will be used.

In [ ]:
# In pandas, we can do this with
manages_df.sort_values(by=['crew_id', 'equip_id'], ascending=[True, False])

,id,crew_id,equip_id
4,5,1,4
2,3,1,3
3,4,2,4
0,1,2,1
1,2,3,2


#### Distinct Values

You can retrieve a unique set of values only. For example, let’s retrieve a list of all ranks that are assigned to our crew members (without any duplicates).

In [ ]:
query_all_ranks = '''
SELECT rank
FROM crew_df
'''
all_ranks_df = duckdb.sql(query_all_ranks).to_df()
all_ranks_df

,rank
0,10
1,9
2,4
3,4
4,1


In [ ]:
query_distinct_ranks = '''
SELECT DISTINCT(rank) AS rank
FROM crew_df
'''
distinct_ranks_df = duckdb.sql(query_distinct_ranks).to_df()
distinct_ranks_df

,rank
0,10
1,9
2,4
3,1


In [ ]:
# In pandas, we can do this with:
pd.DataFrame(crew_df['rank'].unique(), columns=['rank'])

,rank
0,10
1,9
2,4
3,1


#### Null Values

Unless you specify in the schema (e.g. when creating the table), values could take on NULL (except for primary key).

In [ ]:
query_null = '''
SELECT *
FROM crew_df
WHERE role_id IS NULL
'''
null_row_df = duckdb.sql(query_null).to_df()
null_row_df

,id,name,rank,role_id
0,5,Brandon,1,NaN


In [ ]:
# In pandas, we can do this with:
crew_df[crew_df['role_id'].isnull()]

,id,name,rank,role_id
4,5,Brandon,1,NaN


### Relationships

Tables have relationships amongst themselves.

One to One: A record in a table is associated with one and only one record in another table. (Crew members will be assigned only one role )

One to Many: A record in a table is associated with more than one record in another table. (A crew member can have multiple records in worklog entries)

Many to Many: Multiple records in a table are associated with multiple records in another table (Crew members can manage multiple equipments, and equipments can be managed by multiple crew members)

<p align = "center">
<img src = "https://imgur.com/5kbMODk.png" width= "900" align ="center"/>





A primary key is a unique identifier for a row.

By storing a primary key for another table we can reference a row in the “foreign” other table. This reference column is referred to as a foreign key.

<p align = "center">
<img src = "https://imgur.com/TzNsc8F.png" width= "900" align ="center"/>




#### Many to Many in Relational Model

In order to allow multiple relationships for each pair of rows across two tables, we need to have a dedicated table for the relationship itself.

Let’s express the crew “manages” equipment relationship.

<p align = "center">
<img src = "https://imgur.com/oWoxPdH.png" width= "900" align ="center"/>


#### Querying with Relationships


<img src = "https://i.stack.imgur.com/VQ5XP.png" width= "700" align ="center"/>



**(INNER) JOIN**: Returns records that have matching values in both tables

**LEFT (OUTER) JOIN**: Returns all records from the left table, and the matched records from the right table

**RIGHT (OUTER) JOIN**: Returns all records from the right table, and the matched records from the left table

**FULL (OUTER) JOIN**: Returns all records when there is a match in either left or right table

**CROSS JOIN**: Returns all possible combinations of table A and table B (i.e. if A has m rows and B has n rows A CROSS JOIN B would return m x n rows)

**NATURAL JOIN**: Combines rows from 2 or more tables based on the common column(s) between them (think of as a "smarter" pandas merge where you don't have to specify the "on" parameter)

We use the JOIN command to query with relationships. Note that the JOIN key word by itself defaults to INNER JOIN.

Let us visualize our two tables again:

In [ ]:
roles_df

,role_id,name
0,1,captain
1,2,scientist
2,3,engineer
3,4,engineer 2


In [ ]:
crew_df

,id,name,rank,role_id
0,1,Jane,10,1.0
1,2,Dan,9,2.0
2,3,Alex,4,3.0
3,4,Jen,4,4.0
4,5,Brandon,1,NaN


Now what if we want to fetch roles of all crew members

In [ ]:
# Pandas
crewmember_role_pandas_df = crew_df.merge(roles_df, on='role_id')
crewmember_role_pandas_df = crewmember_role_pandas_df.rename(columns={'name_x': 'name', 'name_y': 'role'})
crewmember_role_pandas_df[['name','role']]

,name,role
0,Jane,captain
1,Dan,scientist
2,Alex,engineer
3,Jen,engineer 2


Table and column names can get messy. We can use the AS operator to alias column names and table names in our queries.


In [ ]:
# SQL
query_crewmember_role = '''
SELECT c.name, r.name AS role
FROM crew_df AS c
  JOIN roles_df AS r
  ON c.role_id = r.role_id'''
crewmember_role_df = duckdb.sql(query_crewmember_role).df()
crewmember_role_df

,name,role
0,Jane,captain
1,Dan,scientist
2,Alex,engineer
3,Jen,engineer 2


A lot of the time INNER JOIN will be what you need, but sometimes you might need to use other more inclusive joins, such as LEFT JOIN. This keeps everything in the left table even if there is no match.

An example of when this would be useful is if you have a Customers table and Orders table, and you wanted to join them to see how many orders each customer has. You want to keep all the customers even if they have 0 orders (otherwise they wouldn't show up in the table).

Let's try the same query but with LEFT JOIN to see the members who were not assigned a role.

In [ ]:
# In Pandas: crew_df.merge(roles_df, on='role_id', how='left')
query_crewmember_role = '''
SELECT c.name, r.name AS role
FROM crew_df AS c
  LEFT JOIN roles_df AS r
  ON c.role_id = r.role_id'''
crewmember_role_df = duckdb.sql(query_crewmember_role).df()
crewmember_role_df

,name,role
0,Jane,captain
1,Dan,scientist
2,Alex,engineer
3,Jen,engineer 2
4,Brandon,None


As shown in the Venn Diagram above, we can make the LEFT JOIN more exclusive by ONLY including the instances where there is no match.

In [ ]:
# In Pandas:
#   merged_df = crew_df.merge(roles_df, on='role_id', how='left')
#   merged_df[merged_df['name_y'].isnull()][['name_x']]

query_crewmember_role = '''
SELECT c.name, r.name AS role
FROM crew_df AS c
  LEFT JOIN roles_df AS r
  ON c.role_id = r.role_id
WHERE r.role_id IS NULL'''
crewmember_role_df = duckdb.sql(query_crewmember_role).df()
crewmember_role_df

,name,role
0,Brandon,None


#### Multiple Joins

To query information from our many-to-many relationships, we can use multiple joins.

Find the equipment handled by each crew member:

In [ ]:
manages_df

,id,crew_id,equip_id
0,1,2,1
1,2,3,2
2,3,1,3
3,4,2,4
4,5,1,4


In [ ]:
# Multiple INNER JOINs
query_manyTomany = '''
SELECT C.name AS name, E.name AS equipment
FROM crew_df C
  JOIN manages_df M
    ON C.id = M.crew_id
  JOIN equipment_df E
    ON M.equip_id = E.id
'''

# Crew name from crew_df (C)
# manages_df (M)
# Equipment name from equipment_df (E)

crewmember_equipment = duckdb.sql(query_manyTomany).df()
crewmember_equipment

,name,equipment
0,Dan,Centrifuge
1,Alex,Soldering Station
2,Jane,Notebook
3,Dan,Chemical Z
4,Jane,Chemical Z


#### Aggregate Operation

Counting (SUM), averaging (AVG), minimum (MIN), maximum (MAX).


Number of Crew Members:

In [ ]:
# Pandas
crew_count = crew_df.shape[0]
crew_count

5

In [ ]:
crew_count = duckdb.sql('''
SELECT COUNT(*) AS count
FROM crew_df
''').df()
crew_count

,count
0,5


#### Grouping

How many hours has each crew member worked in total? Let's return crew member name and number of hours, and only keep the crew members that have worked **>15 hours**.


In [ ]:
# Pandas
merged_df = pd.merge(crew_df, worklog_df, how='inner', left_on='id', right_on='crew_id')
grouped_df = merged_df.groupby(['id_x', 'name'], as_index=False)['hours'].sum()
result_df = grouped_df[grouped_df['hours'] > 15]
result_df[['name','hours']]

,name,hours
2,Alex,17
3,Jen,22


The HAVING clause is used to filter the **results** of the GROUP BY (note difference compared to WHERE clause, which filters the **rows**).

In [ ]:
crew_hours = duckdb.sql('''
SELECT c.name, SUM(w.hours) AS hours
FROM crew_df c
  JOIN worklog_df w
    ON c.id = w.crew_id
GROUP BY c.id, c.name
HAVING SUM(w.hours) > 15
''').df()

crew_hours

,name,hours
0,Alex,17.0
1,Jen,22.0


**IMPORTANT**: All columns in the SELECT statement must either appear in the GROUP BY or appear in an aggregation (what happens if we didn't group by `c.name`?).

#### Intermediate Results

For more complex queries, we may find it convenient to calculate intermediate results and then use that in another query. There are 2 ways to do this:


1. **Nested Queries**: Writing another SQL query (i.e. SELECT FROM WHERE...) as a part of an outer clause. Use when the nested query is relatively simple and when its result is only needed once.
2. **Common Table Expressions (CTEs)**: Created at the beginning of the main query, is a temporary, named query result that can be used throughout the main query. Can always be used to make queries more readable; use over nested queries when you need to use a certain intermediate result more than once.



Find the equipments handled by crew members and only return those crew members who are ranked either 4 or 10 and their name contains a letter e.

In [ ]:
# Using nested query

complex1_df = duckdb.sql('''
SELECT cname, equipment_df.name
FROM equipment_df
JOIN (SELECT manages_df.equip_id as eid, crew_df.name AS cname
      FROM crew_df
        JOIN manages_df
          ON crew_df.id = manages_df.crew_id
      WHERE crew_df.name
        LIKE '%e%' AND crew_df.rank IN ('4', '10') )
  ON eid = equipment_df.id
''').df()

complex1_df

,cname,name
0,Alex,Soldering Station
1,Jane,Notebook
2,Jane,Chemical Z


In [ ]:
# Using CTE

complex2_df = duckdb.sql('''
WITH crew_manages_cte AS (
  SELECT manages_df.equip_id AS eid, crew_df.name AS cname
  FROM crew_df
  JOIN manages_df
    ON crew_df.id = manages_df.crew_id
  WHERE crew_df.name LIKE '%e%'
    AND crew_df.rank IN ('4', '10')
)

SELECT crew_manages_cte.cname, equipment_df.name
FROM equipment_df
JOIN crew_manages_cte
  ON crew_manages_cte.eid = equipment_df.id;
''').df()

complex2_df

,cname,name
0,Alex,Soldering Station
1,Jane,Notebook
2,Jane,Chemical Z


#### Some takeaways

**Sequence of execution (top to bottom) :**

**FROM** & **JOINs** determine & filter rows

**WHERE** more filters on the rows

**GROUP BY** combines those rows into groups

**HAVING** filters groups

**SELECT** projects data from a database (columns)

**ORDER BY** arranges the remaining rows/groups

**LIMIT** filters on the remaining rows/groups

## Advanced SQL

SQL is extremely versatile! There are many other SQL functions and expressions that we haven't covered in this lab, but here are a few useful ones to check out:

*   [CASE WHEN](https://www.w3schools.com/sql/sql_case.asp): way to handle if/else logic
*   [Window Functions](https://www.geeksforgeeks.org/window-functions-in-sql/) (RANK(), PARTITION BY, etc.): apply to aggregate and ranking functions over a particular window (set of rows). The OVER clause is used with window functions to define that window.
*   Set Operations ([UNION](https://www.w3schools.com/sql/sql_union.asp), INTERSECT, IN/NOT IN, etc.): combine the results of multiple queries into a single result


